In [ ]:
import os
import cv2
import numpy as np
import networkx as nx
import torch
import torchvision.transforms as transforms
from skimage.segmentation import felzenszwalb
import
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.color import rgb2gray
from tqdm import tqdm

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import skimage.feature as skf

In [ ]:
def GraphSegmentation(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)  # Convert to RGB for display

    # Define mask for GrabCut
    row, col, _ = image.shape
    mask = np.zeros((row, col), np.uint8)

    # Initialize models correctly
    bgdModel = np.zeros((1, 65), np.float64)  # Background model
    fgdModel = np.zeros((1, 65), np.float64)  # Foreground model

    x0, y0 = 1, 1  # Ensure valid rectangle coordinates
    x1, y1 = col , row
    rect = (x0, y0, x1 - x0, y1 - y0)

    # Run GrabCut
    cv.grabCut(image, mask, rect, bgdModel, fgdModel, 10, cv.GC_INIT_WITH_RECT)

    # Modify mask to extract foreground
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")
    segmented = image * mask2[:, :, np.newaxis]
    return segmented
        


In [ ]:
def ExtractFeatures(image):
    edges = cv.Canny(image, 50, 150)
    edge_count = np.sum(edges > 0)
    glcm = skf.graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = skf.graycoprops(glcm, 'contrast')[0, 0]
    correlation = skf.graycoprops(glcm, 'correlation')[0, 0]
    energy = skf.graycoprops(glcm, 'energy')[0, 0]
    homogeneity = skf.graycoprops(glcm, 'homogeneity')[0, 0]
    return [contrast, correlation, energy, homogeneity]




In [ ]:

dataset_path = r"C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\Dataset"  # Replace with actual path
categories = ["Positive", "Negative"]  # Crack = positive, No crack = negative

def process_images():
    data = []
    labels = []
    
    for category in categories:
        label = 1 if category == "positive" else 0  # Assign labels
        folder_path = os.path.join(dataset_path, category)
        
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            segmented=GraphSegmentation(img)
            features=ExtractFeatures(segmented)
        




